# Исследование опроса клиентов телекомунникацонной компании

Импортируем библиотеки и подключимся к БД:

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/_____csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id AS user_id,
        u.lt_day AS lt_day,
        (CASE
        WHEN u.lt_day < 366 THEN "new"
        ELSE "old"
        END) AS is_new,
        u.age AS age,
        (CASE
        WHEN u.gender_segment = 1 THEN "women"
        ELSE "men"
        END) AS gender_segment,
        u.os_name AS os_name, 
        u.cpe_type_name AS cpe_type_name,
        l.country AS country,
        l.city AS city,
        age.title AS age_segment,
        tr.title AS traffic_segment,
        lt.title AS lifetime_segment, 
        u.nps_score AS nps_score,
        (CASE 
        WHEN u.nps_score BETWEEN 9 AND 10 THEN "promoters"
        WHEN u.nps_score BETWEEN 7 AND 8 THEN "passives"
        ELSE "detractors"
        END) AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS age ON u.age_gr_id=age.age_gr_id
LEFT JOIN traffic_segment AS tr ON u.tr_gr_id=tr.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id=lt.lt_gr_id;
"""

Прочитаем файлы

In [4]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,women,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,old,53.0,men,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,old,57.0,men,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,old,44.0,women,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,old,24.0,men,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters
5,A005O0,5501,old,42.0,women,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,detractors
6,A0061R,1236,old,45.0,men,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,promoters
7,A009KS,313,new,35.0,men,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,promoters
8,A00AES,3238,old,36.0,women,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,promoters
9,A00F70,4479,old,54.0,women,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,promoters


Посмотрим явные дубликаты:

In [5]:
df.duplicated().sum()

0

Явных дубликатов не обнаружено. 

Видим, что данные в столбце 'age' представлены в float, а должно быть в int. Я пробовала, кстати, исправить тип уже в самом Tableau еще в первой итерации, но у меня появились Null, поэтому не стала трогать. 

Посмотрим, какие значения есть в столбце `age`:

In [6]:
df['age'].unique()

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., nan, 14., 71., 61.,
       16., 77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85.,
       87., 86.])

In [7]:
df['age'].describe()

count    501939.000000
mean         39.621946
std          11.188249
min          10.000000
25%          31.000000
50%          38.000000
75%          47.000000
max          89.000000
Name: age, dtype: float64

Видим, что есть NaN, а также самый минимальный возраст - 10 лет и максимаальный - 89. Возможно, 10-летний пользователь является "выбросом", так как обычно в опросах обычно участвуют подростки от 14 лет либо взрослые. 

Посмотрим столбец `nps_score` на предмет выбросов:

In [8]:
df['nps_score'].describe()

count    502493.000000
mean          7.508562
std           3.020378
min           1.000000
25%           5.000000
50%           9.000000
75%          10.000000
max          10.000000
Name: nps_score, dtype: float64

Видим, что с данными все хорошо, минимальное значение 1 и максимальное - 10, как и должно быть.

Посмотрим столбец `lt_day`:

In [9]:
df['lt_day'].describe()

count    502493.000000
mean       1868.841439
std        1683.701762
min         -21.000000
25%         533.000000
50%        1239.000000
75%        3064.000000
max        9162.000000
Name: lt_day, dtype: float64

Здесь видим непривычное значение - "-21", что несвойственно для количества месяцев жизни клиента, число которых должно быть положительным. 

Сохраним файл для дальнейшей работы в Tableau

In [10]:
df.to_csv('project_2_sql_tableau.csv', index=False)